# [LVV-T1802] - Integration of M2 Hexapod with SAL

This case will verify that the integration of the M2 Hexapod with SAL.
The blocks below represent the steps of the test case.

Requirements
* EFD
* Hexapod(s) powered on
* Thermal sensors attached to the six actuators of the hexapod.
* CSC running

This test will require manual verification of certain events and telemetry in the summit EFD.
Also manual verification of appropriate temperatures for each actuator.

**Make sure you run this notebook on TTS before running at the summit.**

Please, see the [README] file for the requirements to run this notebook.

[LVV-T1802]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T1802
[README]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/README.md

## Setting Up Test Environment

Before we run the tests, we want to make sure that we have all the libraries imported, remotes connected, etc.

In [ ]:
test_case = "LVV-T1802"
test_exec = "LVV-EXXXX"

In [ ]:
import asyncio
import logging
import os
import yaml

import astropy.units as u
import numpy as np
import pandas as pd

from astropy.time import Time
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

from lsst_efd_client import EfdClient
from lsst.ts import salobj
from lsst.ts.idl.enums import MTHexapod

from lsst.sitcom import vandv

In [ ]:
exec_info = vandv.ExecutionInfo()
print(exec_info)

In [ ]:
client = vandv.efd.create_efd_client()

The following block sets the necessary environment variables for setting up the DDS/SAL communication

In [ ]:
os.environ["LSST_DDS_HISTORYSYNC"] = "30"

This is the amount of time to wait in between a move just to make sure that the actuators are not overheating.  

In [ ]:
STD_WAIT = 39

This sets up the logger for the test.

In [ ]:
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

Starts a Script controller which allows putting custom messages into the EFD for later analysis.

In [ ]:
start_time = datetime.now()
test_message = "M2 Hexapod Integration Test"

script = salobj.Controller("Script", index=42658885)
await asyncio.sleep(10) # May help with DDS problems; closing all other kernels may help too
print(f"{test_case} {test_exec} time to start is {datetime.now() - start_time} [s]")

This is how you start the remote for the CSC.

In [ ]:
domain = salobj.Domain()
print(domain)

Create the remote to controle the M2 Hexapod.  
Remember that the `index` tells if we are running the Camerea Hexapod (`1`) or the M2 Hexapod (`2`). 

In [ ]:
csc_index = 2
hexapod_csc = salobj.Remote(name="MTHexapod", domain=domain, index=csc_index)
print(hexapod_csc)

Start the hexapod service.

In [ ]:
await hexapod_csc.start_task

Check for heartbeats.

In [ ]:
await hexapod_csc.evt_heartbeat.next(flush=True, timeout=5)

---
This test case also uses the Mount and the Rotator.  
Make sure that you have them in the correct configuration (simulator/hardware and ccw-following enabled/disabled).  
The cell below starts them:

In [ ]:
mount = salobj.Remote(name="MTMount", domain=domain)
await asyncio.sleep(10)
await mount.start_task

In [ ]:
rotator = salobj.Remote(name="MTRotator", domain=domain)
await asyncio.sleep(10)
await rotator.start_task

---
Before you run the test, make sure that M2Hex is in the, at least, DISABLED state.  
Otherwise, it will not publish telemetry nor events.  

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.STANDBY)

In [ ]:
e = hexapod_csc.evt_summaryState.get()
print("M2Hexapod now in state:", e.summaryState)

# Summary State 2 = ENABLED
# Summary State 1 = DISABLED
if e.summaryState not in [1, 2]:
    print("  Transitioning to DISABLED state")
    await salobj.set_summary_state(hexapod_csc, salobj.State.DISABLED)
else:
    print("  Nothing to do.")

## Running the test

From any state send the CSC into OfflineState/AvailableState using the EUI.  
Change the control at the EUI to `eGUI` and check that the `MTHexapod_logevent_commandableByDDS` publishes false.

In [ ]:
e = hexapod_csc.evt_commandableByDDS.get()
print(e)

Now, change the control at the EUI to `DDS` and check that the `MTHexapod_logevent_commandableByDDS` publishes true.

In [ ]:
e = hexapod_csc.evt_commandableByDDS.get()
print(e)

Check the transition above using the EFD.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTHexapod.logevent_commandableByDDS", 
    fields="state",
    num=5,
    index=2
)

print(df)

--- 
Verify the TCP/IP is connected to the low level controller.

In [ ]:
e = hexapod_csc.evt_connected.get()
print(e)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTHexapod.logevent_connected", 
    fields="connected",
    num=5,
    index=2
)

print(df)

--- 
Verify the __MTHexapod_logevent_configuration__ event is publishing data to the EFD.

In [ ]:
e = hexapod_csc.evt_configuration.get()
print(e)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTHexapod.logevent_configuration", 
    fields="*",
    num=1,
    index=2
)

print(df)

---
Verify the __MTHexapod_logevent_interlock__ event is unengaged and publishing data to the EFD.

In [ ]:
e = hexapod_csc.evt_interlock.get()
print(e)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTHexapod.logevent_interlock", 
    fields="engaged",
    num=1,
    index=2
)

print(df)

---
Hit the E-stop and verify that the __MTHexapod_logevent_interlock__ event is engaged and publishing data to the EFD.

In [ ]:
e = hexapod_csc.evt_interlock.get()
print(e)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTHexapod.logevent_interlock", 
    fields="engaged",
    num=1,
    index=2
)

print(df)

---
Reset the E-stop and clear the error.  
With the CSC in the FAULT state, issue a command to send the CSC to STANDBY using the notebook.

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.STANDBY)

---
If running at the summit, verify that the thermal sensors are connected and producing telemetry into the EFD in the __lsst.sal.ESS.temperature__ topic.

In [ ]:
# ToDo @b1quint: Improve this query  
df = await client.select_top_n(
    "lsst.sal.ESS.temperature",
    fields="*",
    num=1,
)

print(df)

---
The following steps define what the Jupyter Notebook for this test case implements.  
Executing the Jupyter notebook is the only actual command and control step that needs to be executed.  
Transition the state machine into disabledState to publish telemetry.  

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.DISABLED)

---
Verify the MTHexapod_actuators telemetry is being published to the EFD with the following parameters:
- calibrated
- raw
- timestamp

In [ ]:
t = hexapod_csc.tel_actuators.get()
print(t)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTHexapod.actuators",
    fields="*",
    num=5,
)

# The `index` keyword does not work in the command above. 
#  Using this to filter data from the data-frame.
df = df[df["private_identity"] == "MTHexapod:2"]
print(df.iloc[0])

---
Verify the MTHexapod_application data is being published to the EFD with the following parameters:
- demand
- position
- error

In [ ]:
t = hexapod_csc.tel_application.get()
print(t)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTHexapod.application",
    fields="*",
    num=5,
)

# The `index` keyword does not work in the command above. 
#  Using this to filter data from the data-frame.
df = df[df["private_identity"] == "MTHexapod:2"]
print(df.iloc[0])

---
Verify the MTHexapod_electrical data is being published to the EFD with the following parameters:
- copleyStatusWordDrive
- copleyLatchingFaultStatus
- motorCurrent
- busVoltage

In [ ]:
t = hexapod_csc.tel_electrical.get()
print(t)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTHexapod.electrical",
    fields="*",
    num=5,
)

# The `index` keyword does not work in the command above. 
#  Using this to filter data from the data-frame.
df = df[df["private_identity"] == "MTHexapod:2"]
print(df.iloc[0])

### Helper functions

In [ ]:
def enabled_substate_callback(evt):
    """Print the enabled substate when event is received."""
    print(MTHexapod.EnabledSubstate(evt.enabledSubstate))

In [ ]:
def in_position_callback(evt):
    """Print the in position event when it is received."""
    print(evt.in_position)

In [ ]:
def print_hexapod_position(_hex):
    """Print the current hexapod position"""
    pos = _hex.tel_application.get()

    print("Current hexapod position:")
    print("".join([f"{pos.position[i]:10.2f}" for i in range(6)]))
    print("\n")

### Test _move_ command

Test Sequence #1 - Synchronous Move Commands
With the synchronous button enabled and in `enabled/stationary` state.

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.ENABLED)

Make sure that the hexapod is in `stationary`.

In [ ]:
await hexapod_csc.cmd_stop.set_start()

In [ ]:
e = hexapod_csc.evt_controllerState.get()
enabled_substate_callback(e)

Make sure that we start our test with all the positions set to 0.

In [ ]:
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

---
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. 
Enter in the data at the 39 second mark.

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

---
Send a move command of (500um, -500um, 200um, 0.01deg, -0.015deg, 0deg).  
  
Record the corresponding DDS events that were generated.  
  
The `controllerState.enabledSubstate` goes to `MOVING_POINT_TO_POINT` when the move begins and `STATIONARY` when the move ends.
An `MTHexapod_logevent_inPosition` event is generated when the move is complete.

In [ ]:
# Start test
script.log.info(f"START - {test_message} -- {test_case} {test_exec} Test move command")

In [ ]:
# Print the current position
print_hexapod_position(hexapod_csc)

In [ ]:
# Replace callback function that allows printing the sub-state
hexapod_csc.evt_controllerState.callback = enabled_substate_callback                                

# Move the hexapod
await hexapod_csc.cmd_move.set_start(x=500,y=-500,z=200, u=0.01,v=-0.015,w=0,sync=True)

# Wait 15 seconds
await asyncio.sleep(15.)

# Print the current position
print_hexapod_position(hexapod_csc)

In [ ]:
# End test
script.log.info(f"STOP - {test_message} -- {test_case} {test_exec} Test move command")

Wait to cool down the struts.

In [ ]:
await asyncio.sleep(STD_WAIT)

---
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

### Test _stop_ command

In the enabled/stationary state, send a move command of (x=0um, y=0um, z=5000um, u=0deg, v=0deg, w=0deg)

This block moves the hexapod to a large position but stops it after waiting 3 seconds.

In [ ]:
# Start test
script.log.info(f"START - {test_message} -- {test_case} {test_exec} Test stop command")

# Print the current position
print_hexapod_position(hexapod_csc)

In [ ]:
# Move the hexapod
await hexapod_csc.cmd_move.set_start(x=0, y=0, z=5000, u=0, v=0, w=0, sync=True)

# Wait 3 seconds - @Todo @b1quint: maybe we need shorter time?
await asyncio.sleep(3)

# Send the stop command
await hexapod_csc.cmd_stop.set_start()

# Wait 15 seconds
await asyncio.sleep(15.)

# Print the current position
print_hexapod_position(hexapod_csc)

In [ ]:
# End test
script.log.info(f"STOP - {test_message} -- {test_case} {test_exec} Test stop command")

Wait to cool down the struts.

In [ ]:
await asyncio.sleep(STD_WAIT)

---
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

Move back to the origin before starting a new test.

In [ ]:
await hexapod_csc.cmd_move.set_start(x=0, y=0, z=0, u=0, v=0, w=0, sync=True)

Wait to cool down struts.

In [ ]:
await asyncio.sleep(STD_WAIT)

### Test _setCompensationMode_ command

The `setCompensationMode` expects information from the MTmount(elevation) and MTRotator(rotation).
So we need to check if both are in the ENABLED state (or at least in DISABLED state) and that they are publishing the required data.

In [ ]:
e = mount.evt_summaryState.get()
print(e.private_identity, MTHexapod.ControllerState(e.summaryState))

t = mount.tel_elevation.get()
print(t.private_identity, t.actualPosition)

e = rotator.evt_summaryState.get()
print(e.private_identity, MTHexapod.ControllerState(e.summaryState))

t = rotator.tel_rotation.get()
print(t.private_identity, t.actualPosition)

In enabled/stationary state, send a move command of (x=0um, y=0um, z=800um, u=0deg, v=0deg, w=0deg).  

The hexapod is expected to move to the position (x=0um, y=0um, z=800um, u=0deg, v=0deg, w=0deg) and, since we are moving in synchronous mode, the actuators complete the move at nearly the same time.

In [ ]:
# Start test
script.log.info(f"START - {test_message} -- {test_case} {test_exec} Test setCompensationMode command")

# Are we in compensation mode?
await hexapod_csc.cmd_setCompensationMode.set_start(enable=False)
e = hexapod_csc.evt_compensationMode.get()
print("Compensation mode enable?", e.enabled)
assert e.enabled == False

In [ ]:
# Move the hexapod
await hexapod_csc.cmd_move.set_start(x=0, y=0, z=800, u=0, v=0, w=0, sync=True)

# Wait movement to complete
await asyncio.sleep(15.)

In [ ]:
# Enable compensation mode
await hexapod_csc.cmd_setCompensationMode.set_start(enable=True)
e = hexapod_csc.evt_compensationMode.get()
print("Compensation mode enable?", e.enabled)
assert e.enabled == True

In [ ]:
# Move the hexapod again - it should not move
await hexapod_csc.cmd_move.set_start(x=0, y=0, z=800, u=0, v=0, w=0, sync=True)

# Wait movement to complete
await asyncio.sleep(15.)

In [ ]:
# End test
script.log.info(f"STOP - {test_message} -- {test_case} {test_exec} Test setCompensationMode command")

@bquint ToDo - bring this notebook to this repository

To test the LUT compensation use: hdrass/Camera_Hexapod/hex_diagnostics.ipynb

---
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

Move back to the origin before starting a new test.

In [ ]:
await hexapod_csc.cmd_move.set_start(x=0, y=0, z=0, u=0, v=0, w=0, sync=True)

Wait to cool down struts.

In [ ]:
await asyncio.sleep(STD_WAIT)

### Test _offset_ command

Synchronous Offset and Move Commands.  
In enabled/stationary state, send a move command of `(x=500um, y=800um, z=200um, u=0deg, v=0deg, w=0deg)`

In [ ]:
# Start test
script.log.info(f"START - {test_message} -- {test_case} {test_exec} Test offset command")

# Print the current position
print_hexapod_position(hexapod_csc)

# Move hexapod
await hexapod_csc.cmd_move.set_start(x=500, y=800, z=200, u=0, v=0, w=0, sync=True)

# Wait movement to complete
await asyncio.sleep(10.)

# Print the current position
print_hexapod_position(hexapod_csc)

# Move hexapod using offset
await hexapod_csc.cmd_offset.set_start(x=0,y=0,z=500,u=0,v=0,w=0,sync=True)

# Wait movement to complete
await asyncio.sleep(10.)

# Print the current position
print_hexapod_position(hexapod_csc)

# End test
script.log.info(f"STOP - {test_message} -- {test_case} {test_exec} Test offset command")

---
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

Move back to the origin before starting a new test.

In [ ]:
await hexapod_csc.cmd_move.set_start(x=0, y=0, z=0, u=0, v=0, w=0, sync=True)

Wait to cool down struts.

In [ ]:
await asyncio.sleep(STD_WAIT)

### Test pivot command

Test Sequence `setPivot` and `move` commands.
In enabled/stationary state, send a move command of `(x=2000um, y=-3500um, z=200um, u=0.01deg, v=-0.05deg, w=0.002deg, sync=true)`

In [ ]:
# Start test
script.log.info(f"START - {test_message} -- {test_case} {test_exec} Test set pivot point")

# Print the current position
print_hexapod_position(hexapod_csc)

In [ ]:
# Move the hexapod
await hexapod_csc.cmd_move.set_start(x=2000, y=-3500, z=200, u=0.01, v=-0.05, w=0.002, sync=True)

# Wait movement to complete
await asyncio.sleep(10.)

# Print the current position
print_hexapod_position(hexapod_csc)

In [ ]:
# Set a new Pivot point
await hexapod_csc.cmd_setPivot.set_start(x=0, y=0, z=0)

# Wait movement to complete
await asyncio.sleep(10.)

In [ ]:
# Move hexapod again
await hexapod_csc.cmd_move.set_start(x=2000,y=-3500,z=200,u=0.01,v=-0.05,w=0.002,sync=True)

# Wait movement to complete
await asyncio.sleep(10.)

# Print the current position
print_hexapod_position(hexapod_csc)

In [ ]:
# Set the pivot point back to the original value (z=0.703 m in um)
await hexapod_csc.cmd_setPivot.set_start(x=0, y=0, z=703000)

# Print the current position
print_hexapod_position(hexapod_csc)

# End test
script.log.info(f"STOP- {test_message} -- LVV-T1802 Test set pivot point")

---
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

Move back to the origin before starting a new test.

In [ ]:
await hexapod_csc.cmd_move.set_start(x=0, y=0, z=0, u=0, v=0, w=0, sync=True)

Wait to cool down struts.

In [ ]:
await asyncio.sleep(STD_WAIT)

### Test _configureLimits_ command

In enabled/stationary state, send a `configureLimits` command of `(12000um, -1000um, 1000um, 0.1, -0.1, 0.05)`.

In [ ]:
# Start test
script.log.info(f"START - {test_message} -- {test_case} {test_exec} Test configureLimits")

# Try to set a new configuration - It should fail 
hexapod_csc.cmd_configureLimits.set_start(maxXY=12000, minZ=-1000, maxZ=1000, maxUV=0.1, minW=-0.1, maxW=0.05)

In [ ]:
# Try again with reasonable values
hexapod_csc.cmd_configureLimits.set_start(maxXY=1000, minZ=-1000, maxZ=1000, maxUV=0.1, minW=-0.1, maxW=0.05)

In [ ]:
# Move the hexapod
await hexapod_csc.cmd_move.set_start(x=850, y=0, z=500, u=0, v=0, w=0, sync=True)

# Wait movement to complete
await asyncio.sleep(15.)

# Print the current position
print_hexapod_position(hexapod_csc)

In [ ]:
# Move the hexapod again, the command should be rejected
await hexapod_csc.cmd_move.set_start(x=1200, y=0, z=200, u=0, v=0, w=0, sync=True)

In [ ]:
# Move the hexapod again, the command should be rejected
await hexapod_csc.cmd_move.set_start(x=990, y=990, z=200, u=0, v=0, w=0, sync=True)

In [ ]:
# Move the hexapod again, the command should be rejected
await hexapod_csc.cmd_move.set_start(x=500, y=500, z=200, u=0, v=0.1, w=0.01, sync=True)

# Wait movement to complete
await asyncio.sleep(15.)

# Print the current position
print_hexapod_position(hexapod_csc)

In [ ]:
# Start test
script.log.info(f"STOP - {test_message} -- {test_case} {test_exec} Test configureLimits")

Move back to the origin before starting a new test.

In [ ]:
await hexapod_csc.cmd_move.set_start(x=0, y=0, z=0, u=0, v=0, w=0, sync=True)

### Test configureAcceleration command 

In enabled/stationary state, at a position of `(0, 0, 0, 0, 0, 0)` with the velocity and acceleration values set to their nominal values, send a move command of (0um, 0um, 4900um, 0 deg, 0 deg, 0 deg).

In [ ]:
# Start test
script.log.info(f"START - {test_message} -- {test_case} {test_exec} Test configureAcceleration")

In [ ]:
# todo @bquint - add a callback to track how much time it takes to complete
# Move the hexapod, it should take about 9s
await hexapod_csc.cmd_move.set_start(x=0, y=0, z=4900, u=0, v=0, w=0, sync=True)

# Wait movement to complete
await asyncio.sleep(15.)

In [ ]:
# Update the acceleration, the command should be rejected
await hexapod_csc.cmd_configureAcceleration.set_start(1000)

In [ ]:
# Update the acceleration, the command should be accepted
await hexapod_csc.cmd_configureAcceleration.set_start(100)

In [ ]:
# todo @bquint - add a callback to track how much time it takes to complete
# Move the hexapod, it should take about 13s
await hexapod_csc.cmd_move.set_start(x=0, y=0, z=0, u=0, v=0, w=0, sync=True)

# Wait movement to complete
await asyncio.sleep(15.)

In [ ]:
# Update the acceleration to the nominal value
await hexapod_csc.cmd_configureAcceleration.set_start(500)

In [ ]:
# Start test
script.log.info(f"STOP - {test_message} -- {test_case} {test_exec} Test configureAcceleration")

### Test _configureVelocity_ command

In enabled/stationary state, at a position of `(0, 0, 0, 0, 0, 0)`, send a `configureVelocity` command of (10000, .01, 100, .01).

In [ ]:
# Start test
script.log.info(f"START - {test_message} -- {test_case} {test_exec} Test configureVelocity")

In [ ]:
# Change velocity
await hexapod_csc.cmd_configureVelocity.set_start(xy=100, z=0.01, uv=200, w=0.01)

In [ ]:
# todo @bquint - add a callback to track how much time it takes to complete
# Move the hexapod, it should take ~20s to complete
await hexapod_csc.cmd_move.set_start(x=0, y=0, z=2000, u=0, v=0, w=0, sync=True)

# Wait movement to complete
await asyncio.sleep(25.)

In [ ]:
# Change velocity
await hexapod_csc.cmd_configureVelocity.set_start(xy=100, z=0.01, uv=100, w=0.01)

In [ ]:
# todo @bquint - add a callback to track how much time it takes to complete
# Move the hexapod using the offset, it should take ~40s to complete
await hexapod_csc.cmd_offset.set_start(x=0, y=0, z=2000, u=0, v=0, w=0, sync=True)

# Wait movement to complete
await asyncio.sleep(50.)

## State Transition Tests

The test case contains multiple steps that do not belong to a notebook. I will skip them for now. 

## Endurance Test

In [ ]:
#Endurance test for the M2 hexapod. CAVE: This are very long moves. This can overheat the hexapod see.
#LVV-T1600 for shorter moves

script.log.info(f"START- {test_message} -- LVV-T1802 Endurance Test")
for i in range(100):
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=-5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=-5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=-5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=-5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)


    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=-5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=-5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=-5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=-5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=-5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=-5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=-5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=-5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)


script.log.info(f"STOPP- {test_message} -- LVV-T1802 Endurance Test ")

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.STANDBY) # Transition the CSC to Standby state

In [ ]:
await domain.close() # Close the remote connection